In [4]:
gas_factor = 30

In [7]:
import pandas as pd
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
import numpy as np
import math

# !!! Determinism !!!
np.random.seed(42)

file_list = [("BLS12", "./bls12/one_off_parallel_1000.csv"),
             ("BN", "./bn/one_off_parallel_1000.csv"),
             ("MNT4", "./mnt4/one_off_parallel_1000.csv"),
            ("MNT6", "./mnt6/one_off_parallel_1000.csv")]

def get_dfs(files):
    results = []
    for file in files:
        (name, path) = file;
        df = pd.read_csv(path)
        keep = df[["modulus_limbs", "group_limbs", "run_microseconds"]].copy()
        results.append(keep)
        
    return results

In [8]:
dataframes = get_dfs(file_list)

In [9]:
dataframes[0].head(15)

,modulus_limbs,group_limbs,run_microseconds
0,4,1,1485
1,4,1,1053
2,4,1,1097
3,10,9,7093
4,4,1,1051
5,4,1,1011
6,13,13,12262
7,4,1,1017
8,4,1,1068
9,15,7,17030


In [10]:
dataframes[1].head(15)

,modulus_limbs,group_limbs,run_microseconds
0,4,1,193
1,4,1,111
2,4,1,115
3,7,5,406
4,4,1,105
5,8,15,565
6,4,1,110
7,4,1,156
8,7,5,357
9,4,1,150


In [21]:
def make_averages(df):
    average = df.groupby(['modulus_limbs', "group_limbs"]).mean()
    average["gas"] = average["run_microseconds"].apply(lambda x: gas_factor * math.ceil(x))
    average = average.copy()
    average.reset_index(inplace=True) 
    average.drop("run_microseconds", axis = 1, inplace = True)
    average = average.groupby(['modulus_limbs']).max()
    average = average.copy()
    average.reset_index(inplace=True) 
    average.drop("group_limbs", axis = 1, inplace = True)

    return average

In [22]:
mnt4 = make_averages(dataframes[0])
mnt4.head(25)

,modulus_limbs,gas
0,4,1455
1,5,2250
2,6,3135
3,7,4515
4,8,6435
5,9,8625
6,10,11145
7,11,14130
8,12,18210
9,13,22410


In [23]:
mnt6 = make_averages(dataframes[1])
mnt6.head(25)

,modulus_limbs,gas
0,4,2085
1,5,3855
2,6,4815
3,7,7290
4,8,9855
5,9,14070
6,10,18255
7,11,22095
8,12,29310
9,13,34710


In [25]:
mnt4.to_csv("./mnt4/one_off_results.csv",index = False)
mnt6.to_csv("./mnt6/one_off_results.csv",index = False)